# Fall Detection using SisFall Dataset
Daniela Dias, nMec 98039

In [77]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from sklearn.utils import compute_class_weight
from tensorflow.keras import layers, models, callbacks
import optuna
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [78]:
# Set random seed for reproducibility
np.random.seed(42)

## Avoiding Subject Bias in Train-Test Split

When working with the SisFall dataset (or any dataset where multiple recordings come from the same individuals), it is essential to avoid data leakage caused by random sample splitting. 

If we randomly split the dataset into training and testing sets, we risk placing samples from the same subject in both sets. This introduces what is known as "subject bias" — the model can inadvertently learn personal characteristics or movement patterns of specific individuals rather than learning to generalize fall detection across new, unseen people. This would artificially inflate evaluation metrics (such as accuracy and F1-score), because the model is partially memorizing rather than generalizing.

To address this, we use a subject-wise splitting strategy:  
- We first extract the list of unique subjects.  
- Then we split these subjects into train and test groups.  
- Finally, we assign samples based on the subject to which they belong.

This ensures that the model is evaluated on entirely unseen individuals, simulating real-world scenarios where fall detection must work for new users. It leads to more honest and generalizable performance metrics.

## Deep Learning Preprocessing

This pipeline prepares raw time-series sensor data for deep learning models such as CNNs or LSTMs.

1. Raw sensor data is segmented into fixed-length overlapping windows (e.g., 2 seconds = 400 samples at 200 Hz).
    - Each window is treated as a single sample.
   - The overlap can be adjusted (e.g., 25% overlap).
2. Each window is converted into a 3D tensor: (number of samples, window length, number of channels).
    - For example, with 400-sample windows and 9 channels (3 sensors × 3 axes), the shape should be (N, 400, 9).
3. Labels are assigned to each window based on the most frequent class label within the window.
4. Input windows are normalized using z-score standardization based on the training set.
5. Subject-wise splitting is used to ensure no overlap of subjects between training and testing sets.

In [79]:
# Load the dataset from CSV file
sisfall_data = pd.read_csv('../reduced_sisfall_dataset.csv')

In [80]:
# Check the first few rows of the dataset
sisfall_data.head()

,accel_adxl345_x,accel_adxl345_y,accel_adxl345_z,gyro_itg3200_x,gyro_itg3200_y,gyro_itg3200_z,accel_mma8451q_x,accel_mma8451q_y,accel_mma8451q_z,label,filename,subject
0,0.066406,-0.699219,-0.386719,-1.098633,-30.761719,-21.484375,0.074219,-0.680664,-0.272461,adl,D01_SA01_R01.txt,SA01
1,0.058594,-0.679688,-0.351562,-3.234863,-34.667969,-18.676758,0.046875,-0.659180,-0.248047,adl,D01_SA01_R01.txt,SA01
2,0.003906,-0.687500,-0.316406,-5.126953,-37.414551,-16.540527,-0.001953,-0.652344,-0.215820,adl,D01_SA01_R01.txt,SA01
3,-0.039062,-0.703125,-0.300781,-6.347656,-39.489746,-13.854980,-0.033203,-0.680664,-0.170898,adl,D01_SA01_R01.txt,SA01
4,-0.082031,-0.746094,-0.246094,-7.812500,-41.198730,-11.657715,-0.072266,-0.723633,-0.129883,adl,D01_SA01_R01.txt,SA01


In [81]:
def segment_data(df, window_size, overlap):
    # Calculate the step size based on the window size and overlap
    step = int(window_size * (1 - overlap))

    # Obtain the sensor columns
    sensor_cols = df.columns[:-3]  # Exclude label, filename, subject

    features = []
    labels = []

    for filename, group in df.groupby('filename'):
        group = group.reset_index(drop=True)
        for start in range(0, len(group) - window_size + 1, step):
            window = group.iloc[start:start + window_size]

            # Extract features and labels
            feature = window[sensor_cols].values
            label = window['label'].value_counts().idxmax()  # Most frequent label in the window

            features.append(feature)
            labels.append(label)

    return np.array(features), np.array(labels)

In [82]:
def preprocess_for_deep_learning(df, window_size=800, overlap=0.5):
    # Extract unique subjects
    subjects = df['subject'].unique()

    # Subject-wise split
    train_subjects, test_subjects = train_test_split(subjects, test_size=0.2, random_state=42)

    # Assign samples based on subject
    train_data = df[df['subject'].isin(train_subjects)]
    test_data = df[df['subject'].isin(test_subjects)]

    # Segment data into windows
    X_train, y_train = segment_data(train_data, window_size, overlap)
    X_test, y_test = segment_data(test_data, window_size, overlap)

    # Normalize using training statistics
    mean = X_train.mean(axis=(0, 1), keepdims=True)
    std = X_train.std(axis=(0, 1), keepdims=True)
    X_train = (X_train - mean) / std
    X_test = (X_test - mean) / std

    # Map labels to numeric (binary classification)
    label_map = {'adl': 0, 'fall': 1}
    y_train = np.vectorize(label_map.get)(y_train)
    y_test = np.vectorize(label_map.get)(y_test)

    return X_train, X_test, y_train, y_test

In [83]:
# Preprocess the data
X_train, X_test, y_train, y_test = preprocess_for_deep_learning(sisfall_data)

In [84]:
# Check the shape
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((10001, 800, 9), (10001,), (2482, 800, 9), (2482,))

In [85]:
EPOCHS = 50
BATCH_SIZE = 64
N_TRIALS = 20

## Convolutional Neural Network (CNN)

### Model Architecture

The model architecture consists of:
1. **Convolutional Layers**: These layers apply convolution operations to the input data, allowing the model to learn spatial hierarchies of features.
2. **MaxPooling Layers**: These layers downsample the feature maps, reducing their dimensionality while retaining important features.
3. **Flatten Layer**: This layer flattens the 3D output from the convolutional layers into a 1D vector, preparing it for the fully connected layers.
4. **Dense Layers**: These layers are fully connected layers that learn high-level representations of the data.
5. **Dropout Layers**: These layers randomly set a fraction of the input units to 0 during training, which helps prevent overfitting.
6. **Output Layer**: This layer uses a sigmoid activation function to produce a probability score for the binary classification task (fall vs. no fall).

In [86]:
def build_cnn_model(input_shape, num_classes=1, hp=None):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))

    model.add(layers.Conv1D(filters=hp['filters1'], kernel_size=hp['kernel1'], activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))

    if hp['use_second_conv']:
        model.add(layers.Conv1D(filters=hp['filters2'], kernel_size=hp['kernel2'], activation='relu'))
        model.add(layers.MaxPooling1D(pool_size=2))

    model.add(layers.GlobalAveragePooling1D())

    model.add(layers.Dense(hp['dense_units'], activation='relu'))
    model.add(layers.Dropout(hp['dropout']))
    model.add(layers.Dense(num_classes, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp['lr']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [87]:
def objective(trial):
    hp = {
        'filters1': trial.suggest_categorical('filters1', [16, 32, 64]),
        'kernel1': trial.suggest_categorical('kernel1', [3, 5, 7]),
        'use_second_conv': trial.suggest_categorical('use_second_conv', [True, False]),
        'filters2': trial.suggest_categorical('filters2', [32, 64, 128]),
        'kernel2': trial.suggest_categorical('kernel2', [3, 5]),
        'dense_units': trial.suggest_int('dense_units', 32, 128),
        'dropout': trial.suggest_float('dropout', 0.2, 0.5),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    }

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_tr), y=y_tr)
        class_weight_dict = dict(enumerate(class_weights))

        model = build_cnn_model(X_tr.shape[1:], num_classes=1, hp=hp)

        early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

        model.fit(X_tr, y_tr, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
                  class_weight=class_weight_dict, validation_data=(X_val, y_val),
                  callbacks=[early_stop])

        y_val_pred = (model.predict(X_val).flatten() > 0.5).astype(int)
        scores.append(f1_score(y_val, y_val_pred))

    return np.mean(scores)

In [88]:
# Run Optuna hyperparameter tuning
cnn_study = optuna.create_study(direction='maximize')
cnn_study.optimize(objective, n_trials=3)

[I 2025-05-03 18:38:18,711] A new study created in memory with name: no-name-fe943c2f-8f73-4bc5-9150-cf1f8cb72bb5


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - accuracy: 0.6490 - loss: 0.6243 - val_accuracy: 0.8152 - val_loss: 0.4692
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 71ms/step - accuracy: 0.8118 - loss: 0.4667 - val_accuracy: 0.8242 - val_loss: 0.4261
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step - accuracy: 0.8246 - loss: 0.4293 - val_accuracy: 0.8275 - val_loss: 0.4060
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.8318 - loss: 0.4093 - val_accuracy: 0.8359 - val_loss: 0.3927
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.8304 - loss: 0.4067 - val_accuracy: 0.8392 - val_loss: 0.3824
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.8364 - loss: 0.3930 - val_accuracy: 0.8383 - val_loss: 0.3697
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - accuracy: 0.8316 - loss: 0.3870 - val_accuracy: 0.8410 - val_loss: 0.3635
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - accuracy: 0.8443 - loss: 0.3723 - val_accu

[I 2025-05-03 18:45:35,023] Trial 0 finished with value: 0.8425189054051652 and parameters: {'filters1': 64, 'kernel1': 5, 'use_second_conv': False, 'filters2': 128, 'kernel2': 5, 'dense_units': 77, 'dropout': 0.42960876349393595, 'lr': 0.00025104011446059986}. Best is trial 0 with value: 0.8425189054051652.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - accuracy: 0.7919 - loss: 0.4693 - val_accuracy: 0.8419 - val_loss: 0.3446
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.8435 - loss: 0.3515 - val_accuracy: 0.8566 - val_loss: 0.3178
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.8509 - loss: 0.3274 - val_accuracy: 0.8638 - val_loss: 0.3014
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.8677 - loss: 0.2975 - val_accuracy: 0.8110 - val_loss: 0.3765
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.8580 - loss: 0.3066 - val_accuracy: 0.8788 - val_loss: 0.2832
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.8756 - loss: 0.2800 - val_accuracy: 0.8668 - val_loss: 0.2952
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.8716 - loss: 0.2805 - val_accuracy: 0.8761 - val_loss: 0.2797
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.8821 - loss: 0.2792 - val_accu

[I 2025-05-03 18:49:42,260] Trial 1 finished with value: 0.8918652944571935 and parameters: {'filters1': 64, 'kernel1': 7, 'use_second_conv': True, 'filters2': 64, 'kernel2': 3, 'dense_units': 117, 'dropout': 0.33367165763695084, 'lr': 0.0016489126494194772}. Best is trial 1 with value: 0.8918652944571935.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7517 - loss: 0.5656 - val_accuracy: 0.8269 - val_loss: 0.3995
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8275 - loss: 0.4037 - val_accuracy: 0.8383 - val_loss: 0.3668
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8415 - loss: 0.3694 - val_accuracy: 0.8476 - val_loss: 0.3518
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8476 - loss: 0.3499 - val_accuracy: 0.8476 - val_loss: 0.3370
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8441 - loss: 0.3396 - val_accuracy: 0.8449 - val_loss: 0.3343
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8443 - loss: 0.3388 - val_accuracy: 0.8527 - val_loss: 0.3233
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8519 - loss: 0.3232 - val_accuracy: 0.8518 - val_loss: 0.3187
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8573 - loss: 0.3189 - val_accuracy: 0

[I 2025-05-03 18:51:39,976] Trial 2 finished with value: 0.8722418695612634 and parameters: {'filters1': 16, 'kernel1': 3, 'use_second_conv': False, 'filters2': 128, 'kernel2': 3, 'dense_units': 102, 'dropout': 0.34702082907171894, 'lr': 0.0015051452446995682}. Best is trial 1 with value: 0.8918652944571935.


In [89]:
# Get the best hyperparameters
best_params = cnn_study.best_params
print("Best CNN hyperparameters:", best_params)

Best CNN hyperparameters: {'filters1': 64, 'kernel1': 7, 'use_second_conv': True, 'filters2': 64, 'kernel2': 3, 'dense_units': 117, 'dropout': 0.33367165763695084, 'lr': 0.0016489126494194772}


In [90]:
cnn_model = build_cnn_model(X_train.shape[1:], num_classes=1, hp=best_params)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

early_stop_final = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
cnn_model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
              class_weight=class_weight_dict, validation_split=0.2,
              callbacks=[early_stop_final])

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.8199 - loss: 0.4402 - val_accuracy: 0.7216 - val_loss: 0.5800
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.8594 - loss: 0.3263 - val_accuracy: 0.7881 - val_loss: 0.4341
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.8764 - loss: 0.3003 - val_accuracy: 0.7496 - val_loss: 0.5488
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.8812 - loss: 0.2857 - val_accuracy: 0.7711 - val_loss: 0.4869
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.8812 - loss: 0.2801 - val_accuracy: 0.8391 - val_loss: 0.3527
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.8873 - loss: 0.2663 - val_accuracy: 0.7431 - val_loss: 0.5596
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.8937 - loss: 0.2502 - val_accuracy: 0.6647 - val_loss: 0.8178
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.9120 - loss: 0.2167 - val_accu

In [91]:
# Evaluate on test set
y_pred_prob = cnn_model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [92]:
# Metrics
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_pred_prob)

print("Test Accuracy:", test_accuracy)
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1-score:", test_f1)
print("Test AUC:", test_auc)

Test Accuracy: 0.8509266720386784
Test Precision: 0.8092334494773519
Test Recall: 0.8601851851851852
Test F1-score: 0.8339317773788151
Test AUC: 0.9374782057378349


In [93]:
# Save the model
cnn_model.save('cnn_model.keras')

## LSTM

In [94]:
def build_lstm_model(input_shape, num_classes=1, hp=None):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))

    model.add(layers.LSTM(units=hp['lstm_units'], return_sequences=hp['return_sequences']))
    if hp['return_sequences']:
        model.add(layers.LSTM(units=hp['lstm_units2']))

    model.add(layers.Dropout(hp['dropout']))
    model.add(layers.Dense(hp['dense_units'], activation='relu'))
    model.add(layers.Dense(num_classes, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp['lr']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [95]:
def objective_lstm(trial):
    hp = {
        'lstm_units': trial.suggest_int('lstm_units', 32, 128),
        'return_sequences': trial.suggest_categorical('return_sequences', [True, False]),
        'lstm_units2': trial.suggest_int('lstm_units2', 32, 128),
        'dense_units': trial.suggest_int('dense_units', 32, 128),
        'dropout': trial.suggest_float('dropout', 0.2, 0.5),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    }

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_tr), y=y_tr)
        class_weight_dict = dict(enumerate(class_weights))

        model = build_lstm_model(X_tr.shape[1:], num_classes=1, hp=hp)

        early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

        model.fit(X_tr, y_tr, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
                  class_weight=class_weight_dict, validation_data=(X_val, y_val),
                  callbacks=[early_stop])

        y_val_pred = (model.predict(X_val).flatten() > 0.5).astype(int)
        scores.append(f1_score(y_val, y_val_pred))

    return np.mean(scores)

In [96]:
lstm_study = optuna.create_study(direction='maximize')
lstm_study.optimize(objective_lstm, n_trials=N_TRIALS)

[I 2025-05-03 18:52:28,239] A new study created in memory with name: no-name-9bd6318f-e42e-4ce2-a522-609e3a53742c


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 190s 2s/step - accuracy: 0.7733 - loss: 0.5143 - val_accuracy: 0.7424 - val_loss: 0.5367
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 196s 2s/step - accuracy: 0.8403 - loss: 0.4211 - val_accuracy: 0.8581 - val_loss: 0.3546
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 190s 2s/step - accuracy: 0.8619 - loss: 0.3733 - val_accuracy: 0.8599 - val_loss: 0.3690
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 184s 2s/step - accuracy: 0.8645 - loss: 0.3619 - val_accuracy: 0.8626 - val_loss: 0.3487
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 182s 2s/step - accuracy: 0.8728 - loss: 0.3266 - val_accuracy: 0.8602 - val_loss: 0.3420
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 183s 2s/step - accuracy: 0.8724 - loss: 0.3293 - val_accuracy: 0.8590 - val_loss: 0.3537
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 182s 2s/step - accuracy: 0.8670 - loss: 0.3202 - val_accuracy: 0.8248 - val_loss: 0.4199
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 183s 2s/step - accuracy: 0.8357 - loss: 0.4192 - val_accu

[I 2025-05-03 20:03:57,091] Trial 0 finished with value: 0.8345342578329685 and parameters: {'lstm_units': 97, 'return_sequences': True, 'lstm_units2': 91, 'dense_units': 32, 'dropout': 0.3181285863239921, 'lr': 0.0007615980513009164}. Best is trial 0 with value: 0.8345342578329685.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 231s 2s/step - accuracy: 0.7894 - loss: 0.4827 - val_accuracy: 0.8593 - val_loss: 0.3779
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 239s 2s/step - accuracy: 0.8573 - loss: 0.3788 - val_accuracy: 0.8617 - val_loss: 0.3539
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 239s 2s/step - accuracy: 0.8723 - loss: 0.3407 - val_accuracy: 0.8179 - val_loss: 0.4058
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 240s 2s/step - accuracy: 0.8634 - loss: 0.3433 - val_accuracy: 0.8560 - val_loss: 0.3472
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 241s 2s/step - accuracy: 0.8708 - loss: 0.3163 - val_accuracy: 0.8584 - val_loss: 0.3519
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 241s 2s/step - accuracy: 0.8700 - loss: 0.3298 - val_accuracy: 0.8635 - val_loss: 0.3379
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 241s 2s/step - accuracy: 0.8767 - loss: 0.3497 - val_accuracy: 0.8620 - val_loss: 0.3721
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 242s 2s/step - accuracy: 0.8701 - loss: 0.3494 - val_accu

[I 2025-05-03 22:08:56,906] Trial 1 finished with value: 0.8368896336508979 and parameters: {'lstm_units': 115, 'return_sequences': True, 'lstm_units2': 112, 'dense_units': 44, 'dropout': 0.31341885056335644, 'lr': 0.0008401111281057586}. Best is trial 1 with value: 0.8368896336508979.


...

[I 2025-05-04 11:53:00,101] Trial 17 finished with value: 0.8317127570486282 and parameters: {'lstm_units': 82, 'return_sequences': False, 'lstm_units2': 39, 'dense_units': 78, 'dropout': 0.43991100909481745, 'lr': 0.0015213602932714694}. Best is trial 15 with value: 0.8474837637339929.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.7652 - loss: 0.5387 - val_accuracy: 0.8494 - val_loss: 0.4042
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.8643 - loss: 0.3847 - val_accuracy: 0.8542 - val_loss: 0.3758
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.8551 - loss: 0.3678 - val_accuracy: 0.8653 - val_loss: 0.3487
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.8701 - loss: 0.3248 - val_accuracy: 0.8599 - val_loss: 0.3411
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - accuracy: 0.8626 - loss: 0.3410 - val_accuracy: 0.8611 - val_loss: 0.3356
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - accuracy: 0.8765 - loss: 0.3188 - val_accuracy: 0.8599 - val_loss: 0.3526
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - accuracy: 0.8718 - loss: 0.3166 - val_accuracy: 0.8605 - val_loss: 0.3361
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - accuracy: 0.8711 - loss: 0.3196 - val_accu

[I 2025-05-04 13:12:39,129] Trial 18 finished with value: 0.8414770752344287 and parameters: {'lstm_units': 101, 'return_sequences': False, 'lstm_units2': 64, 'dense_units': 116, 'dropout': 0.2562139336351145, 'lr': 0.0004352744818365048}. Best is trial 15 with value: 0.8474837637339929.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 49s 456ms/step - accuracy: 0.8013 - loss: 0.4796 - val_accuracy: 0.8503 - val_loss: 0.3926
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 49s 467ms/step - accuracy: 0.8508 - loss: 0.3957 - val_accuracy: 0.8572 - val_loss: 0.3594
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 48s 458ms/step - accuracy: 0.8562 - loss: 0.3909 - val_accuracy: 0.8389 - val_loss: 0.4242
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 48s 457ms/step - accuracy: 0.8472 - loss: 0.4239 - val_accuracy: 0.8500 - val_loss: 0.3917
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 48s 453ms/step - accuracy: 0.8594 - loss: 0.3884 - val_accuracy: 0.8539 - val_loss: 0.3811
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 88ms/step
Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 52s 482ms/step - accuracy: 0.7891 - loss: 0.4857 - val_accuracy: 0.8440 - val_loss: 0.3877
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 50s 474ms/step - accuracy: 0.8556 - loss: 0.3943 - val_accuracy: 0.8605 - val_loss: 0.3672
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 5

[I 2025-05-04 13:30:27,402] Trial 19 finished with value: 0.8297208865265951 and parameters: {'lstm_units': 46, 'return_sequences': False, 'lstm_units2': 52, 'dense_units': 99, 'dropout': 0.4479772062626143, 'lr': 0.009277449431024553}. Best is trial 15 with value: 0.8474837637339929.


In [97]:
# Get the best hyperparameters
best_lstm_params = lstm_study.best_params
print("Best LSTM hyperparameters:", best_lstm_params)

Best LSTM hyperparameters: {'lstm_units': 84, 'return_sequences': False, 'lstm_units2': 64, 'dense_units': 114, 'dropout': 0.27012230359446965, 'lr': 0.00048695716869272786}


In [98]:
lstm_model = build_lstm_model(X_train.shape[1:], num_classes=1, hp=best_lstm_params)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

early_stop_final = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lstm_model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
               class_weight=class_weight_dict, validation_split=0.2,
               callbacks=[early_stop_final])

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 78s 608ms/step - accuracy: 0.8052 - loss: 0.5148 - val_accuracy: 0.7006 - val_loss: 0.6298
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 77s 618ms/step - accuracy: 0.8636 - loss: 0.3921 - val_accuracy: 0.7366 - val_loss: 0.5303
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 78s 628ms/step - accuracy: 0.8695 - loss: 0.3660 - val_accuracy: 0.7486 - val_loss: 0.5602
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 71s 569ms/step - accuracy: 0.8787 - loss: 0.3401 - val_accuracy: 0.5802 - val_loss: 0.6360
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 70s 563ms/step - accuracy: 0.8762 - loss: 0.3323 - val_accuracy: 0.7676 - val_loss: 0.4475
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 68s 541ms/step - accuracy: 0.8848 - loss: 0.3110 - val_accuracy: 0.7786 - val_loss: 0.5060
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 70s 560ms/step - accuracy: 0.8795 - loss: 0.3235 - val_accuracy: 0.7971 - val_loss: 0.4251
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 68s 547ms/step - accuracy: 0.8820 - loss: 0

In [99]:
y_pred_prob = lstm_model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

78/78 ━━━━━━━━━━━━━━━━━━━━ 19s 239ms/step


In [100]:
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_pred_prob)

print("LSTM Test Accuracy:", test_accuracy)
print("LSTM Test Precision:", test_precision)
print("LSTM Test Recall:", test_recall)
print("LSTM Test F1-score:", test_f1)
print("LSTM Test AUC:", test_auc)

LSTM Test Accuracy: 0.8686543110394843
LSTM Test Precision: 0.8976793248945147
LSTM Test Recall: 0.787962962962963
LSTM Test F1-score: 0.8392504930966469
LSTM Test AUC: 0.92364875574576


In [101]:
# Save the model
lstm_model.save('lstm_model.keras')

## Hybrid

In [102]:
def build_cnn_lstm_model(input_shape, num_classes=1, hp=None):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))

    model.add(layers.Conv1D(filters=hp['filters'], kernel_size=hp['kernel_size'], activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.LSTM(units=hp['lstm_units']))

    model.add(layers.Dropout(hp['dropout']))
    model.add(layers.Dense(hp['dense_units'], activation='relu'))
    model.add(layers.Dense(num_classes, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp['lr']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [103]:
def objective_cnn_lstm(trial):
    hp = {
        'filters': trial.suggest_categorical('filters', [16, 32, 64]),
        'kernel_size': trial.suggest_categorical('kernel_size', [3, 5, 7]),
        'lstm_units': trial.suggest_int('lstm_units', 32, 128),
        'dense_units': trial.suggest_int('dense_units', 32, 128),
        'dropout': trial.suggest_float('dropout', 0.2, 0.5),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    }

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_tr), y=y_tr)
        class_weight_dict = dict(enumerate(class_weights))

        model = build_cnn_lstm_model(X_tr.shape[1:], num_classes=1, hp=hp)

        early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

        model.fit(X_tr, y_tr, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
                  class_weight=class_weight_dict, validation_data=(X_val, y_val),
                  callbacks=[early_stop])

        y_val_pred = (model.predict(X_val).flatten() > 0.5).astype(int)
        scores.append(f1_score(y_val, y_val_pred))

    return np.mean(scores)

In [104]:
cnn_lstm_study = optuna.create_study(direction='maximize')
cnn_lstm_study.optimize(objective_cnn_lstm, n_trials=N_TRIALS)

[I 2025-05-04 13:45:02,406] A new study created in memory with name: no-name-5b99cae8-3683-4ce6-9460-494455c98f9b


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 81s 755ms/step - accuracy: 0.8159 - loss: 0.4581 - val_accuracy: 0.8524 - val_loss: 0.3951
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 79s 750ms/step - accuracy: 0.8652 - loss: 0.3517 - val_accuracy: 0.8602 - val_loss: 0.3530
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 79s 757ms/step - accuracy: 0.8706 - loss: 0.3435 - val_accuracy: 0.8623 - val_loss: 0.3545
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 79s 756ms/step - accuracy: 0.8667 - loss: 0.3472 - val_accuracy: 0.8641 - val_loss: 0.3474
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 79s 755ms/step - accuracy: 0.8668 - loss: 0.3480 - val_accuracy: 0.8587 - val_loss: 0.3513
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 80s 758ms/step - accuracy: 0.8717 - loss: 0.3276 - val_accuracy: 0.8647 - val_loss: 0.3461
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 79s 753ms/step - accuracy: 0.8748 - loss: 0.3177 - val_accuracy: 0.8620 - val_loss: 0.3474
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 79s 755ms/step - accuracy: 0.8668 - loss: 0

[I 2025-05-04 14:35:39,312] Trial 0 finished with value: 0.8437095788956176 and parameters: {'filters': 16, 'kernel_size': 3, 'lstm_units': 126, 'dense_units': 51, 'dropout': 0.41109753183840114, 'lr': 0.0012203102306550062}. Best is trial 0 with value: 0.8437095788956176.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 33s 293ms/step - accuracy: 0.8095 - loss: 0.4530 - val_accuracy: 0.8587 - val_loss: 0.3621
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 31s 291ms/step - accuracy: 0.8434 - loss: 0.4101 - val_accuracy: 0.8494 - val_loss: 0.3875
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 30s 287ms/step - accuracy: 0.8510 - loss: 0.4018 - val_accuracy: 0.8569 - val_loss: 0.3722
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 30s 288ms/step - accuracy: 0.8588 - loss: 0.3684 - val_accuracy: 0.8551 - val_loss: 0.3532
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 31s 293ms/step - accuracy: 0.8637 - loss: 0.3587 - val_accuracy: 0.8602 - val_loss: 0.3591
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 30s 288ms/step - accuracy: 0.8681 - loss: 0.3497 - val_accuracy: 0.8521 - val_loss: 0.3510
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 31s 295ms/step - accuracy: 0.8578 - loss: 0.3598 - val_accuracy: 0.8401 - val_loss: 0.3708
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 31s 296ms/step - accuracy: 0.8555 - loss: 0

[I 2025-05-04 14:59:41,183] Trial 1 finished with value: 0.846688709658984 and parameters: {'filters': 16, 'kernel_size': 3, 'lstm_units': 70, 'dense_units': 86, 'dropout': 0.3539024431036102, 'lr': 0.007527553876055296}. Best is trial 1 with value: 0.846688709658984.


...

[I 2025-05-04 21:42:23,239] Trial 17 finished with value: 0.8681724090898514 and parameters: {'filters': 32, 'kernel_size': 7, 'lstm_units': 127, 'dense_units': 75, 'dropout': 0.25468436032632863, 'lr': 0.0014723588607452852}. Best is trial 14 with value: 0.8762420417811149.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 50s 459ms/step - accuracy: 0.8203 - loss: 0.4389 - val_accuracy: 0.7663 - val_loss: 0.4232
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 47s 451ms/step - accuracy: 0.8601 - loss: 0.3597 - val_accuracy: 0.8560 - val_loss: 0.3611
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 49s 469ms/step - accuracy: 0.8657 - loss: 0.3388 - val_accuracy: 0.8647 - val_loss: 0.3365
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 48s 458ms/step - accuracy: 0.8639 - loss: 0.3270 - val_accuracy: 0.8605 - val_loss: 0.3520
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 48s 460ms/step - accuracy: 0.8672 - loss: 0.3292 - val_accuracy: 0.8593 - val_loss: 0.3411
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 50s 476ms/step - accuracy: 0.8777 - loss: 0.3204 - val_accuracy: 0.8665 - val_loss: 0.3335
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 54s 515ms/step - accuracy: 0.8730 - loss: 0.3402 - val_accuracy: 0.8659 - val_loss: 0.3592
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 49s 462ms/step - accuracy: 0.8768 - loss: 0

[I 2025-05-04 22:19:15,697] Trial 18 finished with value: 0.8650176027817152 and parameters: {'filters': 64, 'kernel_size': 7, 'lstm_units': 112, 'dense_units': 116, 'dropout': 0.31969190931938646, 'lr': 0.0035013285734162773}. Best is trial 14 with value: 0.8762420417811149.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 41s 369ms/step - accuracy: 0.8276 - loss: 0.4314 - val_accuracy: 0.8563 - val_loss: 0.3650
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 41s 387ms/step - accuracy: 0.8618 - loss: 0.3631 - val_accuracy: 0.8482 - val_loss: 0.3885
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 38s 365ms/step - accuracy: 0.8508 - loss: 0.3896 - val_accuracy: 0.8638 - val_loss: 0.3560
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 39s 368ms/step - accuracy: 0.8646 - loss: 0.3305 - val_accuracy: 0.8596 - val_loss: 0.3553
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 39s 376ms/step - accuracy: 0.8626 - loss: 0.3347 - val_accuracy: 0.8617 - val_loss: 0.3175
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 39s 376ms/step - accuracy: 0.8603 - loss: 0.3301 - val_accuracy: 0.8728 - val_loss: 0.3110
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 39s 373ms/step - accuracy: 0.8585 - loss: 0.3224 - val_accuracy: 0.8605 - val_loss: 0.3253
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 51s 483ms/step - accuracy: 0.8601 - loss: 0

[I 2025-05-04 22:42:28,648] Trial 19 finished with value: 0.8530237340204355 and parameters: {'filters': 64, 'kernel_size': 7, 'lstm_units': 93, 'dense_units': 128, 'dropout': 0.28047844562964597, 'lr': 0.003752590044811205}. Best is trial 14 with value: 0.8762420417811149.


In [105]:
# Get the best hyperparameters
best_cnn_lstm_params = cnn_lstm_study.best_params
print("Best CNN-LSTM hyperparameters:", best_cnn_lstm_params)

Best CNN-LSTM hyperparameters: {'filters': 64, 'kernel_size': 7, 'lstm_units': 93, 'dense_units': 91, 'dropout': 0.2486291360645874, 'lr': 0.002938384347909423}


In [106]:
cnn_lstm_model = build_cnn_lstm_model(X_train.shape[1:], num_classes=1, hp=best_cnn_lstm_params)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

early_stop_final = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
cnn_lstm_model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
                   class_weight=class_weight_dict, validation_split=0.2,
                   callbacks=[early_stop_final])

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 318ms/step - accuracy: 0.8307 - loss: 0.4472 - val_accuracy: 0.8191 - val_loss: 0.2641
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 46s 366ms/step - accuracy: 0.8709 - loss: 0.3654 - val_accuracy: 0.8046 - val_loss: 0.4641
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 339ms/step - accuracy: 0.8788 - loss: 0.3291 - val_accuracy: 0.7811 - val_loss: 0.3742
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 320ms/step - accuracy: 0.8816 - loss: 0.3060 - val_accuracy: 0.7806 - val_loss: 0.4658
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 338ms/step - accuracy: 0.8837 - loss: 0.2838 - val_accuracy: 0.7951 - val_loss: 0.3995
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 332ms/step - accuracy: 0.8844 - loss: 0.2817 - val_accuracy: 0.7681 - val_loss: 0.5579


In [107]:
y_pred_prob = cnn_lstm_model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

78/78 ━━━━━━━━━━━━━━━━━━━━ 11s 141ms/step


In [108]:
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_pred_prob)

print("CNN-LSTM Test Accuracy:", test_accuracy)
print("CNN-LSTM Test Precision:", test_precision)
print("CNN-LSTM Test Recall:", test_recall)
print("CNN-LSTM Test F1-score:", test_f1)
print("CNN-LSTM Test AUC:", test_auc)

CNN-LSTM Test Accuracy: 0.8815471394037067
CNN-LSTM Test Precision: 0.9207708779443254
CNN-LSTM Test Recall: 0.7962962962962963
CNN-LSTM Test F1-score: 0.8540218470705064
CNN-LSTM Test AUC: 0.9173561578697099


In [109]:
# Save the model
cnn_lstm_model.save('cnn_lstm_model.keras')

## Transformer

In [110]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout):
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [111]:
def build_transformer_model(input_shape, num_classes=1, hp=None):
    inputs = layers.Input(shape=input_shape)
    x = inputs

    for _ in range(hp['num_transformer_blocks']):
        x = transformer_encoder(x, head_size=hp['head_size'], num_heads=hp['num_heads'], ff_dim=hp['ff_dim'],
                                dropout=hp['dropout'])

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(hp['dense_units'], activation='relu')(x)
    x = layers.Dropout(hp['dropout'])(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp['lr']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [112]:
def objective_transformer(trial):
    hp = {
        'num_transformer_blocks': trial.suggest_int('num_transformer_blocks', 1, 3),
        'head_size': trial.suggest_categorical('head_size', [32, 64]),
        'num_heads': trial.suggest_categorical('num_heads', [2, 4]),
        'ff_dim': trial.suggest_int('ff_dim', 32, 128),
        'dense_units': trial.suggest_int('dense_units', 32, 128),
        'dropout': trial.suggest_float('dropout', 0.1, 0.5),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    }

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_tr), y=y_tr)
        class_weight_dict = dict(enumerate(class_weights))

        model = build_transformer_model(X_tr.shape[1:], num_classes=1, hp=hp)

        early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

        model.fit(X_tr, y_tr, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
                  class_weight=class_weight_dict, validation_data=(X_val, y_val),
                  callbacks=[early_stop])

        y_val_pred = (model.predict(X_val).flatten() > 0.5).astype(int)
        scores.append(f1_score(y_val, y_val_pred))

    return np.mean(scores)

In [114]:
transformer_study = optuna.create_study(direction='maximize')
transformer_study.optimize(objective_transformer, n_trials=10)

[I 2025-05-04 22:57:13,231] A new study created in memory with name: no-name-7baa2c95-805f-4d2a-9257-db32e292e492


Epoch 1/50
  1/105 ━━━━━━━━━━━━━━━━━━━━ 1:02:10 36s/step - accuracy: 0.4844 - loss: 0.7074

[W 2025-05-04 22:58:33,792] Trial 0 failed with parameters: {'num_transformer_blocks': 3, 'head_size': 64, 'num_heads': 4, 'ff_dim': 119, 'dense_units': 116, 'dropout': 0.387254726692843, 'lr': 0.00866178071813757} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\DanielaDias\Documents\Github\CAA\venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\DanielaDias\AppData\Local\Temp\ipykernel_21800\107445073.py", line 25, in objective_transformer
    model.fit(X_tr, y_tr, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
  File "C:\Users\DanielaDias\Documents\Github\CAA\venv\Lib\site-packages\keras\src\utils\traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\DanielaDias\Documents\Github\CAA\venv\Lib\site-packages\keras\src\backend\tensorflow\trainer.py", li

KeyboardInterrupt: 

In [ ]:
# Get the best hyperparameters
best_transformer_params = transformer_study.best_params
print("Best Transformer hyperparameters:", best_transformer_params)

In [ ]:
transformer_model = build_transformer_model(X_train.shape[1:], num_classes=1, hp=best_transformer_params)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

early_stop_final = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
transformer_model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
                      class_weight=class_weight_dict, validation_split=0.2,
                      callbacks=[early_stop_final])

In [ ]:
y_pred_prob = transformer_model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

In [ ]:
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_pred_prob)

print("Transformer Test Accuracy:", test_accuracy)
print("Transformer Test Precision:", test_precision)
print("Transformer Test Recall:", test_recall)
print("Transformer Test F1-score:", test_f1)
print("Transformer Test AUC:", test_auc)

In [ ]:
# Save the model
transformer_model.save('transformer_model.keras')